# Exploratory Data Analysis of the National Collision Database 2017

In [8]:
import pandas as pd
import numpy as np
import altair as alt

from sklearn.model_selection import train_test_split

alt.data_transformers.enable('data_server')
alt.renderers.enable('mimetype');

In [ ]:
#pip install altair_data_server

## Read in the data set

In [17]:
# Getting list of column names from NCDB
col_names = pd.read_csv('data/NCDB_2017.csv', nrows=0).columns

# Creating custom data type dictionary for NCDB columns
types_dict = {'C_YEAR': int, 'C_SEV': int, 'C_CASE': int}
types_dict.update({col: str for col in col_names if col not in types_dict})

# Importing NCDB
ncdb = pd.read_csv('data/NCDB_2017.csv', dtype=types_dict)

# Creating 'target' column to convert multi-class outcomes to binary-class
ncdb.loc[ncdb['P_ISEV'] == '3', 'TARGET'] = "True"
ncdb.loc[ncdb['P_ISEV'] != '3', 'TARGET'] = "False"

## Summary of the data set

In [22]:
count = pd.DataFrame(ncdb['TARGET'].value_counts())
count.rename(index={'False':'No fatality', 'True':'Fatality'}, columns={'TARGET':'Count'}, inplace=True)
count

,Count
No fatality,287985
Fatality,1856


In [23]:
ncdb.head(5)

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,...,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER,C_CASE,TARGET
0,2017,01,1,10,2,01,02,03,1,3,...,UUUU,01,M,75,11,2,NN,1,2455950,False
1,2017,01,1,12,2,01,04,UU,1,5,...,UUUU,01,F,21,11,2,02,1,2455959,False
2,2017,01,1,00,2,01,03,UU,7,3,...,UUUU,01,F,34,11,2,02,1,2455998,False
3,2017,01,1,17,2,02,21,UU,1,1,...,UUUU,01,F,50,11,2,02,1,2456104,False
4,2017,01,1,17,2,02,21,UU,1,1,...,UUUU,01,M,63,11,1,NN,1,2456104,False


In [24]:
ncdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289841 entries, 0 to 289840
Data columns (total 24 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   C_YEAR  289841 non-null  int64 
 1   C_MNTH  289841 non-null  object
 2   C_WDAY  289841 non-null  object
 3   C_HOUR  289841 non-null  object
 4   C_SEV   289841 non-null  int64 
 5   C_VEHS  289841 non-null  object
 6   C_CONF  289841 non-null  object
 7   C_RCFG  289841 non-null  object
 8   C_WTHR  289841 non-null  object
 9   C_RSUR  289841 non-null  object
 10  C_RALN  289841 non-null  object
 11  C_TRAF  289841 non-null  object
 12  V_ID    289841 non-null  object
 13  V_TYPE  289841 non-null  object
 14  V_YEAR  289841 non-null  object
 15  P_ID    289841 non-null  object
 16  P_SEX   289841 non-null  object
 17  P_AGE   289841 non-null  object
 18  P_PSN   289841 non-null  object
 19  P_ISEV  289841 non-null  object
 20  P_SAFE  289841 non-null  object
 21  P_USER  289841 non-null  object
 

In [25]:
ncdb.describe()

,C_YEAR,C_SEV,C_CASE
count,289841.0,289841.000000,2.898410e+05
mean,2017.0,1.984585,2.512671e+06
std,0.0,0.123198,3.293840e+04
min,2017.0,1.000000,2.455828e+06
25%,2017.0,2.000000,2.484670e+06
50%,2017.0,2.000000,2.512042e+06
75%,2017.0,2.000000,2.541478e+06
max,2017.0,2.000000,2.570235e+06


## Partition the data set into training and test sets

In [6]:

nc_train, nc_test = train_test_split(ncdb, test_size=0.1, random_state=21)
print("The shape of the national collision train data is:", nc_train.shape)
print("The shape of the national collision train data is:", nc_test.shape)

The shape of the national collision train data is: (260856, 24)
The shape of the national collision train data is: (28985, 24)


## Exploratory analysis on the training data set

In [7]:
# brush = alt.selection_interval()

# EDA on the Training dataset
nc_true = (alt.Chart(
    data=nc_train.query("target=='True'"),
    title={"text": ["Collision by Month"],
           "subtitle": ["Year 2017"]}
).mark_bar(
    opacity=0.5
).encode(
    x=alt.X('C_MNTH',
            title='Month',
            axis=alt.Axis(labelAngle=0)),
    y=alt.Y('count()',
            title='Number of collisions'),
    # color=alt.condition(brush,
    #                     alt.Color('target:N', scale=None),
    #                     alt.value('lightgray'))
    color=alt.Color('target',
                    scale=alt.Scale(scheme='oranges'))
# ).add_selection(brush)
)
)

In [8]:
nc_false = (alt.Chart(
    data=nc_train.query("target=='False'"),
    title={"text": ["Collision by Month"],
           "subtitle": ["Year 2017"]}
).mark_bar(
    color="coral",
    opacity=0.5
).encode(
    x=alt.X('C_MNTH',
            title='Month',
            axis=alt.Axis(labelAngle=0)),
    y=alt.Y('count()',
            title='Number of collisions'),
    color=alt.Color('target',
                    scale=alt.Scale(scheme='magma'))
)
)

In [9]:
alt.layer(nc_false, nc_true).resolve_scale(
    y='independent',
    color='independent'
)

alt.LayerChart(...)